In [1]:
### Colab define cell
FOLDER_PATH = '/content/gdrive/MyDrive/Counterfactual-prototype-main/LORE/LORE-master'

try:
    import google.colab
    RUNNING_ON_COLAB = True
except: 
    RUNNING_ON_COLAB = False

if RUNNING_ON_COLAB: 
    import sys
    from google.colab import drive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials 
    drive.mount('/content/gdrive', force_remount=True)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    gdrive = GoogleDrive(gauth)
    sys.path.append(FOLDER_PATH)

In [2]:
## test the original code here.
## sudo apt-get install libtbb2 // needed for ubuntu

import lore

from lore.prepare_dataset import *
from lore.neighbor_generator import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


# dataset_name = 'german_credit.csv'
path_data = './lore/datasets/'
# dataset = prepare_german_dataset(dataset_name, path_data)

dataset_name = 'compas-scores-two-years.csv'
dataset = prepare_compass_dataset(dataset_name, path_data)
print(dataset['label_encoder'][dataset['class_name']].classes_)
print(dataset['possible_outcomes'])

# dataset_name = 'adult.csv'
# dataset = prepare_adult_dataset(dataset_name, path_data)

X, y = dataset['X'], dataset['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

blackbox = RandomForestClassifier(n_estimators=20)
blackbox.fit(X_train, y_train)

X2E = X_test
y2E = blackbox.predict(X2E)
y2E = np.asarray([dataset['possible_outcomes'][i] for i in y2E])

idx_record2explain = 0

explanation, infos = lore.explain(idx_record2explain, X2E, dataset, blackbox,
                                  ng_function=genetic_neighborhood,
                                  discrete_use_probabilities=True,
                                  continuous_function_estimation=False,
                                  returns_infos=True,
                                  path=path_data, sep=';', log=False)

dfX2E = build_df2explain(blackbox, X2E, dataset).to_dict('records')
dfx = dfX2E[idx_record2explain]
# x = build_df2explain(blackbox, X2E[idx_record2explain].reshape(1, -1), dataset).to_dict('records')[0]

print('x = %s' % dfx)
print('r = %s --> %s' % (explanation[0][1], explanation[0][0]))
for delta in explanation[1]:
    print('delta', delta)

covered = lore.get_covered(explanation[0][1], dfX2E, dataset)
print(len(covered))
print(covered)

print(explanation[0][0][dataset['class_name']], '<<<<')

def eval(x, y):
    return 1 if x == y else 0

precision = [1-eval(v, explanation[0][0][dataset['class_name']]) for v in y2E[covered]]
print(precision)
print(np.mean(precision), np.std(precision))


['High' 'Medium-Low']
['Medium-Low', 'High']


/home/jrhs/.pyenv/versions/3.8.11/envs/tf/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/jrhs/.pyenv/versions/3.8.11/envs/tf/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


x = {'class': 'Medium-Low', 'age': 28, 'age_cat': '25 - 45', 'sex': 'Male', 'race': 'African-American', 'priors_count': 5, 'days_b_screening_arrest': 0, 'c_charge_degree': 'F', 'is_recid': 1, 'is_violent_recid': 0, 'two_year_recid': 0, 'length_of_stay': 51}
r = {'age': '>25', 'priors_count': '<=8', 'length_of_stay': '>44'} --> {'class': 'Medium-Low'}
delta {}
73
[0, 3, 16, 82, 97, 145, 175, 185, 188, 223, 241, 300, 316, 319, 326, 389, 406, 425, 437, 451, 478, 501, 511, 526, 531, 586, 595, 602, 607, 617, 638, 645, 650, 682, 698, 701, 719, 733, 809, 842, 846, 857, 859, 870, 871, 890, 891, 893, 898, 899, 908, 919, 937, 941, 948, 953, 985, 999, 1009, 1060, 1092, 1122, 1183, 1208, 1253, 1324, 1340, 1363, 1366, 1368, 1384, 1414, 1432]
Medium-Low <<<<
[1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1]
0.726027397260274 0.44599